In [195]:
from scipy.signal import find_peaks
from sklearn import preprocessing
import statistics

ekg,features=load_ekg("training/chapman_shaoxing/g1/JS00066")

def beat_characteristics(ekg, lead_num=1):
    """
    RR_var: variance in distance between beats (distance between peaks)
    HR: Heart Rate
    """
    scaler = preprocessing.MinMaxScaler(feature_range=(0, 1))
    
    lead = ekg[lead_num][1000:4000]    
    all_peaks, _ = find_peaks(lead, height=max(lead)/1.4, distance=150)

    intervals = np.diff(all_peaks)
    intervals_normalized = scaler.fit_transform(np.reshape(intervals,(-1,1)))

    RR_var = statistics.variance(intervals)
    RR_var_normalized = statistics.variance(np.reshape(intervals_normalized,(1,len(intervals_normalized)))[0])
    HR = 30000/statistics.mean(result)
    
    return HR, RR_var, RR_var_normalized

def is_invalid(ekg):
    for lead_num in range(12):
        HR, RR_var, RR_var_normalized = beat_characteristics(ekg,lead_num)
        if RR_var >100000:
            return True
    return False
    

In [ ]:
def remove_baseline_wander_old(signal):
    """
    Removes baseline wander from all leads, takes nd-array as input
    """
    proc_signal = np.ndarray((0, signal.shape[1]))
    for x in signal:
        ssds = np.zeros((3))

        cur_lp = np.copy(x)
        iterations = 0
        while True:
            # Decompose 1 level
            lp, hp = dwt(cur_lp, "db4")

            # Shift and calculate the energy of detail/high pass coefficient
            ssds = np.concatenate(([np.sum(hp ** 2)], ssds[:-1]))

            # Check if we are in the local minimum of energy function of high-pass signal
            if ssds[2] > ssds[1] and ssds[1] < ssds[0]:
                break

            cur_lp = lp[:]
            iterations += 1

        # Reconstruct the baseline from this level low pass signal up to the original length
        baseline = cur_lp[:]
        for _ in range(iterations):
            baseline = idwt(baseline, np.zeros((len(baseline))), "db4")
        new = x - baseline[: len(x)]
        proc_signal = np.vstack((proc_signal, new))
    return proc_signal